In [ ]:
!pip install clip

  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-0.2.0-py3-none-any.whl size=6989 sha256=f57801fa59e5ab4c576eb95827314bcb611ced0c7a722fc41f63e2fbd7f14014
  Stored in directory: /root/.cache/pip/wheels/7f/5c/e6/2c0fdb453a3569188864b17e9676bea8b3b7e160c037117869
Successfully built clip


In [ ]:
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
from transformers import CLIPProcessor, CLIPModel
from torchvision.datasets import CIFAR10
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "openai/clip-vit-base-patch32"  # or any other CLIP model
model = CLIPModel.from_pretrained(model_name).to(device)
processor = CLIPProcessor.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

cifar10_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)

Files already downloaded and verified


In [ ]:
def zero_shot_classification(image, model, processor, class_names):

    text_inputs = [f"This is a photo of a {class_name}" for class_name in class_names]
    inputs = processor(text=text_inputs, images=image, return_tensors="pt", padding=True).to(device)


    with torch.no_grad():
        outputs = model(**inputs)

    #similarities
    logits_per_image = outputs.logits_per_image
    probs = logits_per_image.softmax(dim=1).cpu().numpy()

    return np.argmax(probs, axis=1)


In [ ]:
# Test on a single image
image, label = cifar10_dataset[0]
image = Image.fromarray(image.permute(1, 2, 0).numpy(), 'RGB')

predicted_class_idx = zero_shot_classification(image, model, processor, class_names)
print(f"Predicted: {class_names[predicted_class_idx[0]]}, Actual: {class_names[label]}")

Predicted: bird, Actual: frog


In [ ]:
correct_predictions = 0
total_images = len(cifar10_dataset)

for i in range(total_images):
    image, label = cifar10_dataset[i]
    image = Image.fromarray(image.permute(1, 2, 0).numpy(), 'RGB')
    predicted_class_idx = zero_shot_classification(image, model, processor, class_names)
    if predicted_class_idx[0] == label:
        correct_predictions += 1

accuracy = correct_predictions / total_images
print(f"Accuracy: {accuracy * 100}%")


Accuracy: 10.696%
